In [1]:
import pandas as pd 
import numpy as np
import time
import datetime
import logging
from selenium import  webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
import sqlalchemy
import warnings
df=pd.read_excel(r"D:\Start\Penalities\Governmental penalties\platenumbers.xlsx",sheet_name='Sheet3')
warnings.filterwarnings('ignore')
driver=webdriver.Chrome()
driver.get('https://ppo.gov.eg/webcenter/portal/PPOPortal/pages_publicservices/trafficservices')


In [5]:
import sqlite3
#file=r'D:\Fleat Data\Report\ahmed\penalty2.db'
exist=[]
notexist=[]
phone=[]
j_t=[]
rr=[]
express=[]
notregister=[]
datanotadd=[]
from datetime import datetime
today=datetime.today().strftime(format='%d-%m-%Y')
def enterplatenumber(platnumber):
    a='no'
    while a=='no':
        try:
            driver.find_element(By.ID,'ixT:NootrfPar022365:LP1::content').clear()
            driver.find_element(By.ID,'ixT:NootrfPar022365:LP2::content').clear()
            driver.find_element(By.ID,'ixT:NootrfPar022365:LP3::content').clear()
            driver.find_element(By.ID,'ixT:NootrfPar022365:LP4::content').clear()
            x=driver.find_element(By.ID,'ixT:NootrfPar022365:LP1::content')
            x.click()
            actions = ActionChains(driver)
            actions.move_to_element(x).send_keys(platnumber).perform()
            time.sleep(3)
            WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH ,'//*[contains(text(),"اجمالي المخالفات")]'))).click()
            a='yes'
            return check(platnumber)
        except Exception as e:
            time.sleep(3)
            pass
def check(platnumber):
    time.sleep(2)
    try:
        print(WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[contains(text(),"البيانات الأساسية للرخصة الحالية")]'))).text)
        print('done')
        time.sleep(3)
        try:
            y=driver.find_element(By.ID,'ixT:NootrfPar022365:it3::content').get_attribute('placeholder')
            if y=='السجل التجاري':
                exist.append(platnumber)
                return commercialregister(platnumber)
            else:

                WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),"رجوع")]')))[0].click()
                phone.append(platnumber)
        except:
            WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),"رجوع")]'))).click()
            phone.append(platnumber)
    except:
        print(WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[contains(text(),"خطأ")]'))).text)
        try :
            error=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH,'//*[contains(text(),"برجاء المحاولة لاحقا")]')))[0].text
            print(error)
            driver.refresh()
            time.sleep(3)
            return check(platnumber)
        except:
            driver.refresh()
            notexist1=[]
            notexist1.append({'scrapping date':today,'plate number':platnumber})
            notexist=pd.DataFrame(notexist1)
            conn = "mssql+pyodbc://./fleat?driver=SQL+Server"
            engine = sqlalchemy.create_engine(conn)
            notexist.to_sql('Weeklynopenalty',con=engine,if_exists='append',index=False)
def commercialregister(platnumber):
    try:
        print('helllo')
        driver.find_element(By.ID,'ixT:NootrfPar022365:it3::content').send_keys('171351')
        code='171351'
        dropdown=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.ID,"ixT:NootrfPar022365:it5::content")))[0]
        select = Select(dropdown)
        select.select_by_index(1)
        try:
            WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="ixT:NootrfPar022365:commandButton4"]/a/span'))).click()
        except:
            WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="ixT:NootrfPar022365:commandButton2"]/a/span'))).click()
        try:
            time.sleep(3)
            WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),"موافق")]'))).click()
            time.sleep(3)
        except:
            pass
        try:
            try:
                driver.find_element(By.XPATH,'//*[contains(text(),"بطاقة ائتمانية")]').text
                print('hello sharawy')
                go=[]
                go.append({'scrapping date':today,'plate number':platnumber})##########3
                nopenalty=pd.DataFrame(go)
                conn = "mssql+pyodbc://LAPTOP-572HGKLU\FLEET/FleatData?driver=SQL+Server"
                engine = sqlalchemy.create_engine(conn)
                nopenalty.to_sql('Weeklynopenalty',con=engine,if_exists='append',index=False)
                engine.dispose()
                WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),"رجوع")]'))).click()
                time.sleep(2)
                WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),"رجوع")]'))).click()
                print('nopenalty')
            except:
                time.sleep(3)
                driver.find_element(By.XPATH,'//*[contains(text(),"تاريخ الواقعة")]').text
                j_t.append(platnumber)
                return add_data(platnumber,code)
            try:
                time.sleep(3)
                WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),"موافق")]'))).click()
                time.sleep(3)
            except:
                pass
        except:
            try:
                driver.refresh()
                time.sleep(3)
                driver.find_element(By.ID,'ixT:NootrfPar022365:it3::content').clear()
                driver.find_element(By.ID,'ixT:NootrfPar022365:it3::content').send_keys('176776')
                code='176776'
                try:
                    WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="ixT:NootrfPar022365:commandButton4"]/a/span'))).click()
                    print('hellllllllllllo')
                except:
                    WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="ixT:NootrfPar022365:commandButton2"]/a/span'))).click()
                time.sleep(3)
                try:
                    time.sleep(3)
                    WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),"موافق")]'))).click()
                    time.sleep(3)
                except:
                    pass
                try:
                    driver.find_element(By.XPATH,'//*[contains(text(),"بطاقة ائتمانية")]').text
                    go=[]
                    go.append({'scrapping date':today,'plate number':platnumber})##########3
                    nopenalty=pd.DataFrame(go)
                    conn = "mssql+pyodbc://LAPTOP-572HGKLU\FLEET/FleatData?driver=SQL+Server"
                    engine = sqlalchemy.create_engine(conn)
                    nopenalty.to_sql('Weeklynopenalty',con=engine,if_exists='append',index=False)
                    engine.dispose()
                    WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),"رجوع")]'))).click()
                    time.sleep(2)
                    WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),"رجوع")]'))).click()
                    print('nopenalty')
                except:
                    try:
                        driver.find_element(By.XPATH,'//*[contains(text(),"بطاقة ائتمانية")]').text
                        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),"رجوع")]'))).click()
                        go=[]
                        go.append({'scrapping date':today,'plate number':platnumber})##########3
                        nopenalty=pd.DataFrame(go)
                        conn = "mssql+pyodbc://LAPTOP-572HGKLU\FLEET/FleatData?driver=SQL+Server"
                        engine = sqlalchemy.create_engine(conn)
                        nopenalty.to_sql('Weeklynopenalty',con=engine,if_exists='append',index=False)
                        engine.dispose()
                        time.sleep(2)
                        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),"رجوع")]'))).click()
                        print('nopenalty')
                    except:
                        driver.find_element(By.XPATH,'//*[contains(text(),"تاريخ الواقعة")]').text
                        rr.append(platnumber)
                        return add_data(platnumber,code)
            except:
                try:
                    driver.refresh()
                    time.sleep(3)
                    driver.find_element(By.ID,'ixT:NootrfPar022365:it3::content').clear()
                    driver.find_element(By.ID,'ixT:NootrfPar022365:it3::content').send_keys('176973')
                    code='176973'
                    try:
                        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="ixT:NootrfPar022365:commandButton4"]/a/span'))).click()
                    except:
                        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="ixT:NootrfPar022365:commandButton2"]/a/span'))).click()
                    time.sleep(3)
                    try:
                        time.sleep(3)
                        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),"موافق")]'))).click()
                        time.sleep(3)
                    except:
                        pass
                    try:
                        driver.find_element(By.XPATH,'//*[contains(text(),"بطاقة ائتمانية")]').text
                        go=[]
                        go.append({'scrapping date':today,'plate number':platnumber})##########3
                        nopenalty=pd.DataFrame(go)
                        conn = "mssql+pyodbc://LAPTOP-572HGKLU\FLEET/FleatData?driver=SQL+Server"
                        engine = sqlalchemy.create_engine(conn)
                        nopenalty.to_sql('Weeklynopenalty',con=engine,if_exists='append',index=False)
                        engine.dispose()
                        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),"رجوع")]'))).click()
                        time.sleep(2)
                        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),"رجوع")]'))).click()
                        print('nopenalty')
                    except:
                        driver.find_element(By.XPATH,'//*[contains(text(),"تاريخ الواقعة")]').text
                        express.append(platnumber)
                        return add_data(platnumber,code)
                except:
                    WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),"رجوع")]'))).click()
                    notregister.append(platnumber)
                    pass
    except:
        driver.find_element(By.ID,'ixT:NootrfPar022365:it3::content').clear()
        
def new(platnumber):
    time.sleep(3)
    a='no'
    while a=='no':   
        try:
            WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),"رجوع")]'))).click()
        except:
            pass
        try:
            time.sleep(2)
            driver.find_element(By.ID,'ixT:NootrfPar022365:cil1::text').click()
        except:
            pass
        try:
            time.sleep(2)
            driver.find_element(By.ID,'ixT:NootrfPar022365:ci0l1::text').click()
        except:
            pass
        a='yes'
def add_data(platnumber,code):
    driver.refresh()
    for i in range(2,200,2):
        try:
            data=[]
            time.sleep(6)
            date=driver.find_element(By.XPATH, '//tbody/tr['+str(i)+']/td[3]').text
            location=driver.find_element(By.XPATH, '//tbody/tr['+str(i)+']/td[4]').text
            description=driver.find_element(By.XPATH, '//tbody/tr['+str(i)+']/td[5]/table/tbody/tr/td[1]').text
            min_amount=driver.find_element(By.XPATH, '//tbody/tr['+str(i)+']/td[5]/table/tbody/tr/td[2]').text
            max_amount=driver.find_element(By.XPATH, '//tbody/tr['+str(i)+']/td[5]/table/tbody/tr/td[3]').text
            amount=driver.find_element(By.XPATH, '//tbody/tr['+str(i)+']/td[5]/table/tbody/tr/td[4]').text
            total_amount=driver.find_element(By.XPATH,'//*[@id="ixT:NootrfPar022365:xxpg26"]/table/tbody/tr[3]/th[5]/table/tbody/tr/th[2]').text
            data.append({'scrapping date':today,'plate number':platnumber,'date':date,'location':location,'description':description,'amount':amount,'total_amount':total_amount,'company code':code,'min amount':min_amount,'max amount':max_amount})
            mydata=pd.DataFrame(data)
            conn = "mssql+pyodbc://LAPTOP-572HGKLU\FLEET/Fleat?driver=SQL+Server"
            engine = sqlalchemy.create_engine(conn)
            mydata.to_sql('AbdoPenalties83',con=conn,if_exists='append',index=False)
            engine.dispose()
            print(platnumber,'          data added')
        except Exception as e:
            return new(platnumber)
            break
for platnumber in df['plate number'].iloc[232:]:
    print(platnumber)
    enterplatenumber(platnumber)

ادف2136
البيانات الأساسية للرخصة الحالية
done
helllo
ادف2136           data added
ادف2136           data added
ادف2136           data added
ادف2136           data added
ادف2136           data added
ادف2136           data added
ادف2136           data added
ادف2136           data added
ادف2136           data added
ادف2136           data added
ادف2136           data added
اسه3648
البيانات الأساسية للرخصة الحالية
done
helllo
اسه3648           data added
اسه3648           data added
اسه3648           data added
اسه3648           data added
اسه3648           data added
اسه3648           data added
اسه3648           data added
اسه3648           data added
اسه3648           data added
اسه3648           data added
اسه3648           data added
اسه3648           data added
اسه3648           data added
اسه3648           data added
اسه3648           data added
اسه3648           data added
اسه3648           data added
اسه3648           data added
اسه3648           data added
اسه3648           data a

In [4]:
df['plate number'].iloc[232:]

232    ادف2136
233    اسه3648
234     صمد851
235    اسو3648
236    اصي5647
237     دون111
238    طقب2686
239    طقب2676
240    طقب2656
241    طقب2682
242    طقب2672
243    جنا1967
244    طهط2631
245    طهط2641
246    طهط2653
247    طهط2657
248    طهط2659
249    الر3238
250    الر3242
251    الر3239
Name: plate number, dtype: object

In [3]:
df[df['plate number']=='ادف2136']


,plate number,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,اجل3653
232,ادف2136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
ادف4916

,plate number
0,ارن8284
1,ارن8287
2,ارن8281
3,ارن8283
4,ارن8292
...,...
681,ادف4167
682,اطف2871
683,اطف2891
684,اطف2873


In [ ]:
اسن9654           data added
